# 线性回归

线性回归是一种单层神经网络，输入有**n**个神经元，输出有**1**个神经元

### 线性回归的关键式子

1. 线性模型的预测公式
$$
\widehat{y} = Xw + b
$$
这个公式表示向量 $\widehat{y}$（预测值）是矩阵 $X$（特征）和向量 $w$（权重）的乘积加上偏置项 $b$。
2. 每个样本的平方误差损失
$$
loss^{i}(w,b) = \frac{1}{2}(w^{T}x^{i} + b - y^{i})^{2}
$$

3. 整个数据集的损失
$$
Loss(w,b) = \frac{1}{n} \sum_{i = 1}^{n} \frac{1}{2}(w^{T}x^{i} + b - y^{i})^{2}
$$

4. 理论上最优参数（$w^{*},b^{*}$）的求解
$$
(w^{*},b^{*}) = argmin_{w,b}Loss(w,b)
$$

### 随机梯度下降方法求解线性回归问题

1. 指定超参数，本问题中是批量大小和学习率。
2. 初始化模型参数的值，如从均值为0、标准差为0.01的正态分布中随机采样，偏置参数初始化为零。
3. 从数据集中随机抽取小批量样本且在负梯度的方向上更新参数，并不断迭代这⼀步骤。
权重更新过程的数学表达如下：

$$
w \leftarrow w - \frac{\eta}{|B|} \sum_{i \in {B}} \frac{\partial{loss^{i}(w,b)}}{\partial{w}} = w - \frac{\eta}{|B|} \sum_{i \in {B}} x^{i}(w^{T}x^{i} + b - y^{i})
$$

$$
b \leftarrow b - \frac{\eta}{|B|} \sum_{i \in {B}} \frac{\partial{loss^{i}(w,b)}}{\partial{b}} = b - \frac{\eta}{|B|} \sum_{i \in {B}} (w^{T}x^{i} + b - y^{i})
$$

> 注意这里是均方误差+简单的线性回归才能手动推导出导数和优化公式，如果是深层神经网络，权重和偏置存在多个需要嵌套计算导数就很难手动推导表达，尤其是还要加入激活函数模块，这时候就开始以**反向传播**作为主要解决方案了

In [1]:
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
from torch import nn

def synthetic_data(w, b, num_examples):  
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w))) # 生成1000个2维数据（1000*2），每个数值为（0，1）的正态分布，因为参数w是2个维度，所以训练数据也要两个维度
    y = torch.matmul(X, w) + b # 矩阵相乘
    y += torch.normal(0, 0.01, y.shape) #每个y加上一点点噪声
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)


def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

# 定义一个线性层
net = nn.Sequential(nn.Linear(2, 1))

# 初始化模型参数
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

# 计算均方误差使用的是MSELoss类，也称为平方L2范数
loss = nn.MSELoss()

# 实例化SGD
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

# 评估
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

epoch 1, loss 0.000194
epoch 2, loss 0.000102
epoch 3, loss 0.000102
w的估计误差： tensor([-2.1458e-04, -8.8930e-05])
b的估计误差： tensor([-0.0006])
